# EP Tender : simulation par chaînes de Markov

## Description du modèle

On considère une station comportant $f$ files, pour un total de $N$ tenders en circulation. Chaque tender comporte $d + 1$ états de charge, numérotés de 0 à $d$. À 0 un tender est complètement déchargé, et à $d$ il est complètement chargé.

On utilise un modèle à temps discret.

Les voitures clientes arrivent de sorte que l'écart temporel entre la $i$-ième voiture et la $(i+1)$-ième est de loi géométrique de paramètre $\lambda$. Lorsqu'elles arrivent elles doivent prendre un tender complètement chargé qu'elles ramènent complètement déchargé au début de l'étape suivante.

Fatalement, un jour plus de $N$ voitures arriveront en même temps et alors la demande ne pourra pas être satisfaite. On note $T_\mathrm{bug}$ le premier temps où la demande en Tender est insatisfaite et on s'intéresse à son espérance. On cherche à trouver, pour un processus de rangement et de sélection des tenders donné, l'espérance de $T_\mathrm{bug}$, qu'on souhaite maximal.

Chaque étape temporelle se déroule comme suit :
1. Tous les tenders dans la station gagnent un niveau de charge, pour un niveau maximum de $d$.
2. Les tenders partis à l'étape précédente sont rangés dans la station.
3. Les nouvelles voitures clientes arrivent et des tenders chargés leur sont attribués.
4. Si la demande a pu être satisfaite, on passe au temps suivant.

## Importation des modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Encodage des états de la station

On remarque tout d'abord qu'on ne peut accéder aux tenders les plus loins dans une file qu'une fois le premier tender parti : il devra donc avoir été entièrement rechargé, et tous ceux arrivés antérieurement le seront de même. Les seules informations pertinentes pour une file sont donc le nombre de tenders présents, et le niveau de charge du plus récemment arrivé.  
  
On encodera donc dans la suite les stations sous la forme de matrice de taille $f \times 2$, où la première colonne désigne le nombre de tenders dans chaque station et la seconde colonne le niveau de charge du tender le plus récemment arrivé pour chaque file.  
Comme on peut permuter les files sans affecter la capacité à servir la demande, on se restreint aux états pour lesquels les files sont de longueurs décroissantes (et, au sein des files de même longueur, de niveau de charge décroissant).

## Charge des tenders

L'algorithme de charge s'écrit simplement dans le cas où les files chargent tous les tenders en même temps.

In [ ]:
def chargeSimplifie(station, d) :
    # Renvoie la même station simplifiée après que tous les tenders aient été chargés de 1
    
    station = station.copy()
        
    for i in range(station.shape[0]) :
        if station[i, 0] != 0 and station[i, 1] != d :
            station[i, 1] += 1
    
    return station

## Liste des stations

On peut désormais lister l'ensemble des états possibles pour les stations.

### Parcours d'ensembles de suite dans l'ordre lexicographique décroissant

Dans la suite, nous allons avoir besoin de fonctions élémentaires permettant de parcourir, dans l'ordre lexicographique décroissant, l'ensemble des suites vérifiant des contraintes données.

In [ ]:
def suiteSuivanteDecSumCst(l) :
    # Permet de parcourir l'ensemble des suites décroissantes de longueur et somme données, selon l'ordre lexicographique
    # décroissant. Renvoie False quand on a atteint le dernier état.
    
    l = l.copy()
    i = len(l) - 1
    curSum = 1
    
    while i >= 0 and curSum > (l[i] - 1) * (len(l) - 1 - i):
        curSum += l[i]
        i -= 1
    
    if i >= 0 :
        l[i] -= 1
        j = i + 1
        while curSum > 0 :
            l[j] = min([curSum, l[i]])
            curSum -= l[j]
            j += 1
        while j < len(l) :
            l[j] = 0
            j += 1
    else :
        l = False
    
    return l    

### D'un état à l'autre, avec les tailles de file fixées

In [ ]:
def etatSimplifieSuivant(station, d) :
    # Passe d'un état simplifié à l'autre, à nombre de tenders par file donné, selon
    # l'ordre lexicographique décroissant pour les niveaux de charge
    
    station = station.copy()
    
    curFile = station.shape[0] - 1
    
    while curFile >= 0 :
        if station[curFile, 1] >= 1 :
            station[curFile, 1] -= 1
            j = curFile + 1
            while j < station.shape[0] and station[curFile, 0] == station[j, 0] :
                station[j, 1] = station[curFile, 1]
                j += 1
            break
        elif station[curFile, 0] != 0 :
            station[curFile, 1] = d
        curFile -= 1
        
    if curFile == -1 :
        station = False
    
    return station

### D'un état à l'autre

In [ ]:
def stationSimplifieeSuivante(station, d) :
    # Permet de parcourir l'ensemble des états possibles pour une station sous forme
    # simplifiee avec un nombre donné de tenders, sans répéter deux états équivalents.
    # Parcourt la répartition des tenders par file selon l'ordre lexicographique
    # décroissant.
    
    station = station.copy()
    
    if type(etatSimplifieSuivant(station, d)) != bool :
        station = etatSimplifieSuivant(station, d)
    else :
        tenderParFile = station[:, 0]
        if type(suiteSuivanteDecSumCst(tenderParFile)) == bool :
            station = False
        else :
            tenderParFile = suiteSuivanteDecSumCst(tenderParFile)
            station[:, 0] = tenderParFile
            for i in range(station.shape[0]) :
                if tenderParFile[i] != 0 :
                    station[i, 1] = d
                else :
                    station[i, 1] = 0
    
    return station

### Liste des stations

In [ ]:
def listAllStations(nTenders, nFiles, d) :
    
    listStations = []
    
    station = np.zeros([nFiles, 2])
    station[0, 0] = nTenders
    station[0, 1] = d

    while type(station) != bool :
        listStations.append(station)
        station = stationSimplifieeSuivante(station, d)
    
    listStations.append(False) # Indique erreur
    
    return listStations

In [ ]:
listAllStations(2, 3, 1)

In [ ]:
listAllStations(4, 2, 2)

## Rangement d'une station désordonnée

Le rangement de tenders dans une file peut conduire la station à ne plus avoir ses files dans l'ordre décroissant. Il faut donc être capable de remettre la station dans l'ordre.

In [ ]:
def rangeStationSimplifiee(station) :
    
    station = station.copy()
    
    maxCharge = max(station[:, 1])
    sortingTable = (maxCharge + 1) * station[:, 0] + station[:, 1]
    
    newIndices = np.argsort(sortingTable)
    
    station = station[newIndices[::-1], :]
    
    return station

## Algorithmes de décision

Il nous faut maintenant décider des algorithmes de décision pour le rangement et l'attribution des tenders. On peut modéliser ces algorithmes par des fonctions $f$ qui prennent en entrée l'état de la station, $S$, et renvoient $f(S)$, l'état de la station après y avoir ajouté le tender déchargé. Plusieurs tenders peuvent arriver en même temps, avec notre modèle, on devrait donc avoir une fonction $f(S, k)$ où $k$ est le nombre de tenders qui arrivent à une étape temporelle donnée, et la valeur prise est l'état de la station après y avoir rangé tous les tenders. Cependant, en pratique, on ne voit arriver qu'une voiture à la fois, et on n'anticipe pas quand les prochaines voitures arriveront, on décide donc de manière itérative : $f(f(S, k), 1) = f(S, k + 1)$ et on n'a donc besoin de connaître que $f(S) = f(S, 1)$.

### Algorithme glouton

On code ici l'algorithme glouton détaillé dans le rapport.

In [ ]:
def rangementGloutonSimplifie(station) :
    
    station = station.copy()
    
    curFile = station.shape[0] - 1
    
    if station[curFile, 0] == 0 :
        station[curFile, 0] += 1
    else :
        minCharge = min(station[:(curFile+1), 1])
        while station[curFile, 1] != minCharge :
            curFile -= 1
        station[curFile, 1] = 0
        station[curFile, 0] += 1
        
    station = rangeStationSimplifiee(station)
    
    return station

In [ ]:
def donGloutonSimplifie(station, d) :
    
    station = station.copy()
    
    curFile = station.shape[0] - 1
    
    while curFile >= 0 and station[curFile, 1] != d :
        curFile -= 1
    
    if curFile == -1 :
        station = False
    else :
        station[curFile, 0] -= 1
        if station[curFile, 0] == 0 :
            station[curFile, 1] = 0
    
        station = rangeStationSimplifiee(station)
    
    return station

### Algorithmes basés sur un indice

On code ici l'algorithme basé sur une classe d'indice, comme détaillé dans la section 6 du rapport. Le choix de la file d'où l'on extrait les tenders reste le même que pour l'algorithme glouton, car ce choix est optimal.

In [ ]:
def rangementIndiceSimplifie(station, indice) :
    
    station = station.copy()
    
    fileRangement = np.argmin([indice(station[station.shape[0] - 1 - i, :]) for i in range(station.shape[0])])
    station[station.shape[0] - 1 - fileRangement, 0] += 1
    station[station.shape[0] - 1 - fileRangement, 1] = 0
        
    station = rangeStationSimplifiee(station)
    
    return station

### Indices

In [ ]:
def indiceLineaire(file, lbd) :
    return lbd * file[1] + file[0]

In [ ]:
def indiceLogarithmique(file, lbd) :
    if file[0] != 0 and file[1] != 0 : 
        v = np.log(lbd) * file[1] + np.log(file[0])
    else :
        v = - np.inf
    return v

In [ ]:
def indiceMultiplicatif(file, lbd) :
    return file[1] * file[0]**lbd

In [ ]:
def indiceMultiplicatifCorr(file, lbd, d) :
    if file[1] == d :
        return np.inf
    else :
        return file[0]**lbd / (d - file[1])

In [ ]:
def indicePuis(file, lbd, mu) :
    return file[1]**lbd + mu * file[0]

In [ ]:
def indicePuisCorr(file, lbd, mu, d) :
    return mu * file[0] - (d - file[1]) ** lbd

In [ ]:
def indicePuisLog(file, lbd, mu) :
    if file[0] == 0 :
        return - np.inf
    else :
        return file[1]**lbd + mu * np.log(file[0])

In [ ]:
def indicePuisLogCorr(file, lbd, mu, d) :
    if file[0] == 0 :
        return - np.inf
    else :
        return mu * np.log(file[0]) - (d - file[1])**lbd

## Création de la matrice de transition

Pour rappel, l'écart entre la date d'arrivée de la $i$-ième voiture et celle de la $(i+1)$-ième suit une loi géométrique de paramètre $\lambda$. Pour simplifier, on change (de manière équivalente) le déroulement de chaque étape temporelle comme suit :
1. Les nouvelles voitures clientes arrivent et des tenders chargés leur sont attribués.
2. Tous les tenders dans la station gagnent un niveau de charge, pour un niveau maximum de $d$.
3. Les tenders partis reviennent et sont rangés dans la station.
4. Si la demande a pu être satisfaite, on passe au temps suivant.

Pour obtenir une chaîne de Markov irréductible, on crée un nouvel état, False, qui dénote que la demande n'a pas pu être satisfaite, et à partir de cet état on repart vers un état arbitraire. On cherche donc le temps moyen pour atteindre False.

In [ ]:
rng = np.random.default_rng(1964)

In [ ]:
def createTransitionMatrix(listeEtat, ajout, retrait, charge, theta) :
    
    transitionMatrix = np.zeros([len(listeEtat)] * 2)
    
    for i in range(len(listeEtat) - 1) :
        curStation = listeEtat[i]
        nClients = 0
        curProb = 1 - theta
        
        while type(curStation) != bool :
            ghostStation = charge(curStation)
            for k in range(nClients) :
                ghostStation = ajout(ghostStation)
            j = 0
            while not (listeEtat[j] == ghostStation).all() :
                j += 1
            transitionMatrix[i, j] += curProb # Même si on reviendra pas deux fois au même état
            curProb *= theta
            nClients += 1
            curStation = retrait(curStation)
        
        transitionMatrix[i, len(listeEtat) - 1] = 1 - np.sum(transitionMatrix[i, :])
    
    transitionMatrix[len(listeEtat) - 1, :] = 1 / len(listeEtat)
        
    return transitionMatrix

In [ ]:
def expectedHittingTime(M, i, j) :
    # Returns expected hitting time from i to j using transition matrix M
    
    M = M.copy()
    M[j, :] = 0
    
    b = np.array([1] * M.shape[0])
    b[j] = 0
    
    v = np.linalg.solve(np.identity(M.shape[0]) - M, b)
    
    return v[i], v

In [ ]:
def plotExpectedHittingTime(nTenders, nFiles, nCharge, ajout, retrait, charge, res = 0.05, etatDepart = None, etatArrivee = None) :
    
    if type(etatDepart) != np.ndarray :
        etatDepart = [[(nTenders // (nFiles - 1)) + 1, nCharge]] * (nTenders % (nFiles - 1))
        etatDepart += [[(nTenders // (nFiles - 1)), nCharge]] * (nFiles - 1 - (nTenders % (nFiles - 1)))
        etatDepart += [[0, 0]]
        etatDepart = np.array(etatDepart)
    
    if type(etatArrivee) != np.ndarray :
        etatArrivee = False
    
    
    listeEtat = listAllStations(nTenders, nFiles, nCharge)    
    ajout = rangementGloutonSimplifie
    retraitBis = lambda x : retrait(x, nCharge)
    chargeBis = lambda x : chargeSimplifie(x, nCharge)
    
    i = 0
    j = 0
    
    while not np.array(listeEtat[i] == etatDepart).all() :
        i += 1
    
    while not np.array(listeEtat[j] == etatArrivee).all() :
        j += 1
    
    
    thetaArray = np.arange(res, 1, res)
    expectedHittingTimes = []
    
    for theta in thetaArray :
        M = createTransitionMatrix(listeEtat, ajout, retraitBis, chargeBis, theta)
        expectedHittingTimes.append(expectedHittingTime(M, i, j)[0])

    return thetaArray, expectedHittingTimes